In [30]:
# -----------------------------------------------
#                  LIBRARIES
# -----------------------------------------------

import geopandas as gpd 
import numpy as np
import pandas as pd 
from fiona.drvsupport import supported_drivers
import requests
import json

# ------------------------------
#       DATA TREATMENT
# ------------------------------

# IMPORT KML DRIVER

supported_drivers['KML'] = 'rw'

# READ KML file to a geopandas dataframe 

geo_df = gpd.read_file('CentrosSecundariaMadrid.kml',driver='KML')

# Create Pandas Dataframe from GeoPandas 

df= pd.DataFrame(geo_df)

f = open("api_key.json",)
api_key = json.load(f)['ors_api_key']

In [2]:
df_xy = geo_df['geometry'].get_coordinates()

In [3]:
df = pd.concat([df, df_xy], axis=1)

In [4]:
df.drop(df[df['x'] == 0].index, inplace = True)

In [5]:
direccion = input()

 Mayor 34, Algete


In [31]:
#REQUEST API DIRECCION

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/geocode/search?api_key={}&text={}&boundary.country=ES'.format(api_key,direccion), headers=headers)

print(call.status_code, call.reason)

data_dir = json.loads(call.text)

200 OK


In [7]:
data_dir['features'][0]['geometry']['coordinates']

[-3.507326, 40.59644]

In [8]:
coord_dir = data_dir['features'][0]['geometry']['coordinates']

In [9]:
coord_ies = [[x,y] for x,y in zip(df['x'],df['y'])]

In [10]:
N_mitad = len(coord_ies)//2

In [11]:
input_api_ors = [coord_dir] + coord_ies

In [12]:
len(input_api_ors)

650

In [32]:
#REQUEST API 0-325 DURACIONES
body = {"locations":input_api_ors[:N_mitad],"destinations":[0]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key,
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)

data = json.loads(call.text)
data['durations'].pop(0)
duraciones_1 = data['durations']

200 OK


In [14]:
#REQUEST API 326-649 DURACIONES

body = {"locations":[coord_dir] + input_api_ors[N_mitad:],"destinations":[0]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248575a3f109d984b2c9918802842d75ae9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)

data = json.loads(call.text)
data['durations'].pop(0)
duraciones_2 = data['durations']

200 OK


In [15]:
duraciones = duraciones_1 + duraciones_2

In [16]:
len(duraciones)

649

In [17]:
df['Duraciones'] = duraciones

In [18]:
df['Duraciones'] = df['Duraciones'].apply(lambda x: x[0])

In [19]:
from humanfriendly import format_timespan

df['Tiempo'] = [format_timespan(i) for i in df['Duraciones']]

In [20]:
df_out = df.sort_values(['Duraciones'])

In [21]:
# Extracting components
df_out['D.A.T.'] = df_out['Description'].str.split('<br>').str[0].str.split(': ').str[1]
df_out['codigo distrito'] = df_out['Description'].str.split('<br>').str[1].str.split(': ').str[1]
df_out['nombre distrito'] = df_out['Description'].str.split('<br>').str[2].str.split(': ').str[1]
df_out['codigo centro'] = df_out['Description'].str.split('<br>').str[3].str.split(': ').str[1]
df_out['lengua'] = df_out['Description'].str.split('<br>').str[4].str.split(': ').str[1]
df_out['nombre centro'] = df_out['Description'].str.split('<br>').str[5].str.split('; ').str[1].str.split(': ').str[1]
df_out['distrito'] = df_out['Description'].str.split('<br>').str[5].str.split('; ').str[0].str.split(': ').str[1]

In [22]:
df_out = df_out.drop(['Description','geometry','Duraciones','codigo distrito', 'lengua', 'distrito','nombre centro'],axis=1)

In [23]:
df_out = df_out[['Tiempo', 'Name', 'nombre distrito','codigo centro','D.A.T.','x','y']]

In [25]:
df_out.to_csv("Resultados/tiempos_{}.csv".format(direccion),index=False)